# Office location selection

For this project we will determine the best posible location for an office that satisfies all the requirements stated in the project description. Here's a summary:

1. It must be near tech startups that have raised more than 1MM USD.
2. It must be near other companies that do design.
3. It must be near schools since 1/3 of employees have at least 1 child.
4. It must be near an airport.
5. It should be near a Starbucks.
6. It should have nearby restaurants with vegan options.
7. It should have a basketball court within 10 Km.
8. It must have a pet grooming service nearby for the company dog.

For this, we will use the companies Mongo DB provided by crunchbase.

First, lets run our python script that defines the functions we will work with:

In [472]:
%run -i 'python_scripts/mongo_connection.py'

Now lets establish the connection to our Mongo Database. For this we will use a function specified in the mongo_connection.py script.

In [473]:
c = connect_mongo("ironhack","companies")
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

Our connection was succesful! Now lets query our companies collection for tech startups and design companies. Again we will use functions defined for these tasks in the mongo_connection.py script since the queries are quite long.

In [474]:
tech = get_tech_startups(1000000,2008)

Query returned  274  companies.


In [475]:
design = get_design_companies()

Query returned  965  companies.


Now that we have our company data, lets process it to eliminate those without coordinates and transform these lists into DataFrames. For this we will use pre-defined functions in our data_process script, lets run it:

In [476]:
%run -i 'python_scripts/data_process.py'


In [477]:
tech_data = get_offices_location(tech)

Received data for  333  companies. 
 202  companies with full location information left.


In [478]:
design_data = get_offices_location(design)

Received data for  1094  companies. 
 708  companies with full location information left.


Now that we have data for all tech and design companies that fit our criteria, lets make a heatmap of their location to see which cities have more of both. For this we will load our mappying script since it has functions that will help us map this information.

In [479]:
%run -i 'python_scripts/mapping.py'

In [480]:
map_heatmap(tech_data, design_data)

Browsing this heatmap, it looks like our offices should be either in San francisco, Los Angeles, New york, Miami, Paris or Manchester. Let's get GeoJsons for these cities.

Rather than looking for defined poligons with the borders of these cities, I created GeoJson files of circles encompassing these cities and their surrounding area. I made them wide enough so that they covered nearby towns that also have important amounts of companies.

In [481]:
city_names = ["san_francisco", "los_angeles", "new_york", "miami", "paris", "manchester"]
cities = load_cities(city_names)
cities

{'San_Francisco': <POLYGON ((-122.241 38.23, -122.312 38.227, -122.383 38.219, -122.452 38.205...>,
 'Los_Angeles': <POLYGON ((-118.113 34.81, -118.207 34.807, -118.3 34.795, -118.392 34.776, ...>,
 'New_York': <POLYGON ((-73.977 41.103, -74.021 41.102, -74.064 41.097, -74.107 41.089, -...>,
 'Miami': <POLYGON ((-80.271 26.121, -80.305 26.12, -80.338 26.115, -80.371 26.108, -8...>,
 'Paris': <POLYGON ((2.354 49.066, 2.322 49.065, 2.292 49.062, 2.261 49.057, 2.232 49....>,
 'Manchester': <POLYGON ((-2.244 53.593, -2.263 53.593, -2.282 53.591, -2.3 53.589, -2.318 ...>}

Now that we have our GeoJsons loaded, lets add the city name to our DataFrames, based on these GeoJsons.

This stepp will check for every office in our Data Frame, if that location is in one of the cirlces defined by the GeoJsons, it will add that city name to our DataFrame.

In [482]:
tech_data = add_city_name(tech_data, cities)
# We will also be adding a "tech" identifier in a new column called "type". This will be useful in a future step, where we will consolidate all of our data in a single DataFrame.
tech_data["type"] = "tech"
tech_data.head()

,name,office description,office latitude,office longitude,city,type
0,Movirtu,Headquarter,51.549971,-0.181600,None,tech
2,Movirtu,India Office,28.582120,77.326699,None,tech
3,GitHub,,37.775196,-122.419204,San_Francisco,tech
5,Gridstore,Gridstore,37.418907,-122.088429,San_Francisco,tech
6,Kluster,New York Office,40.723652,-73.985281,New_York,tech


Now we will do the same thing for our desgin companies

In [483]:
design_data = add_city_name(design_data, cities)
design_data["type"] = "design"
design_data.head()

,name,office description,office latitude,office longitude,city,type
0,Technorati,,37.779558,-122.393041,San_Francisco,design
1,AddThis,HQ - Virginia,38.926172,-77.245195,None,design
2,AddThis,New York Office,40.724604,-73.996876,New_York,design
3,AddThis,Los Angeles Office,34.026302,-118.380954,Los_Angeles,design
6,AddThis,Michigan Office,42.557958,-83.167884,None,design


Now lets join these DataFrames and find out the top cities with more companies.

In [484]:
full_data = pd.concat([tech_data, design_data], axis=0)
top_cities = full_data.groupby(['city', 'type'])['name'].agg('count').reset_index()
top_cities

,city,type,name
0,Los_Angeles,design,46
1,Los_Angeles,tech,14
2,Manchester,design,3
3,Miami,design,7
4,New_York,design,80
5,New_York,tech,22
6,Paris,design,10
7,Paris,tech,4
8,San_Francisco,design,123
9,San_Francisco,tech,58


Looks like our top 3 cities with more tech and design companies are San Francisco, New York and Los Angeles. Lets drop the rest:

In [485]:
top_cities = top_cities.drop([2,3,6,7])
top_cities

,city,type,name
0,Los_Angeles,design,46
1,Los_Angeles,tech,14
4,New_York,design,80
5,New_York,tech,22
8,San_Francisco,design,123
9,San_Francisco,tech,58


Having narrowed down our search to these 3 cities lets map them.
First, lets consolidate our data:

In [486]:
final_data = full_data[full_data['city'].isin(['Los_Angeles', 'New_York', 'San_Francisco'])]
final_data.head()

,name,office description,office latitude,office longitude,city,type
3,GitHub,,37.775196,-122.419204,San_Francisco,tech
5,Gridstore,Gridstore,37.418907,-122.088429,San_Francisco,tech
6,Kluster,New York Office,40.723652,-73.985281,New_York,tech
17,Meez,,37.785271,-122.397582,San_Francisco,tech
18,Gamook,None,37.419240,-122.208414,San_Francisco,tech


Now lets map our 3 cities:

In [487]:
top_3_map = top_3_map(final_data)
top_3_map

Browsing this map, lets focus on San Francisco since it is the city with most companies.

In [488]:
top_3_map.fit_bounds([[37.8,-122.4],[37.5,-122.1]], padding=(4,4))
top_3_map

Let's find out the point that is closest to all of our companies. I'll calculate the mean coordinate for our tech companies and for our design companies separately and add this to our map:

In [489]:
sf_middle_tech = mean_coordinates(final_data[(final_data["city"] == 'San_Francisco')&(final_data["type"] == 'tech')])
sf_middle_design = mean_coordinates(final_data[(final_data["city"] == 'San_Francisco')&(final_data["type"] == 'design')])

add_marker("sf_tech_center","orange","computer",sf_middle_tech,top_3_map)
add_marker("sf_design_center","red","fa-shopping-bag",sf_middle_design,top_3_map)


It looks like our centers lie in the middle of the bay. This is due to the fact that SF has a city center and nearby cities also have a lot of tech companies. Both of these areas surround the San Francisco Bay.

Lets split our analysis in two, the SF city center and southern nearby cities.

First, lets get new GeoJsons and find out many companies are in each sub divison.

In [490]:
city_names = ["sf_center", "sf_south"]
sf_cities = load_cities(city_names)
sf_cities

{'Sf_Center': <POLYGON ((-122.419 37.883, -122.432 37.883, -122.445 37.881, -122.457 37.87...>,
 'Sf_South': <POLYGON ((-122.406 37.569, -122.355 37.518, -122.3 37.486, -122.241 37.448,...>}

Let's create a new DF just for SF center and SF south

In [491]:
sf_center = final_data[final_data['city'] == "San_Francisco"]
sf_south = sf_center

From these new data sets we will asign new city names based on their location and our new disctintion.

In [492]:
sf_center = add_city_name(sf_center, sf_cities)
sf_south = add_city_name(sf_south, sf_cities)
sf_center = sf_center[sf_center["city"]=="Sf_Center"]
sf_center.sample(5)

/Users/kike/Documents/Personal/Educación/IronHack/Module 1/Week 4/project-III-geospatial-data/python_scripts/data_process.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['city'] = city_names


,name,office description,office latitude,office longitude,city,type
3,GitHub,,37.775196,-122.419204,Sf_Center,tech
39,Pixelpipe,World Headquarters,37.764395,-122.401024,Sf_Center,tech
302,GoingOn,"GoingOn Networks, Inc.",37.782263,-122.392142,Sf_Center,tech
386,AdParlor,San Francisco,37.788660,-122.401406,Sf_Center,design
242,InnerRewards,West Coast,37.786910,-122.404403,Sf_Center,tech


In [493]:
sf_south = sf_south[sf_south["city"]=="Sf_South"]
sf_south.sample(5)

,name,office description,office latitude,office longitude,city,type
188,ScaleMP,,37.322973,-122.038579,Sf_South,tech
203,Relevad,Silicon Valley Office,37.559267,-122.302251,Sf_South,design
696,Fanminder,Headquarters,37.379249,-122.115127,Sf_South,design
80,iCharts,Main Office,37.384239,-122.012475,Sf_South,tech
141,PLYmedia,,37.400821,-122.131383,Sf_South,design


Now lets find out the center for both categories and both cities and map them all. Lets start a fresh new map for this:

In [494]:
sf_map = sf_map(sf_center, sf_south)
sf_map

Great! Looks like both sectors have mean coordinates pretty close to each other.
Let's find out which one has more companies:

In [495]:
sf = pd.concat([sf_center, sf_south], axis=0)
sf.groupby(["city","type"])["name"].agg("count").reset_index()

,city,type,name
0,Sf_Center,design,67
1,Sf_Center,tech,26
2,Sf_South,design,45
3,Sf_South,tech,25


Since the San Francisco City Center has more companies and they are all closer together, let's focus on this area for our location.

Now we will find out the mean coordinates between the centers of both sectors. This will define the most desireable location for our office.

In [496]:
sf_center_tech_mean = mean_coordinates(sf_center[(sf_center["type"] == 'tech')])
sf_center_design_mean = mean_coordinates(sf_center[(sf_center["type"] == 'design')])

office_location = mean_coordinates_raw([sf_center_tech_mean,sf_center_design_mean])
office_location

[37.7824, -122.4039]

Great! Now that we have our desired location lets see how it sits on the map in relation to our other companies

In [497]:
sf_map_2 = sf_map_2(sf_center, office_location)
sf_map_2

Awesome! Looks like our office is right in the center of San Francisco.

Now lets take into consideration the rest of the requirements:
1. Near airports
2. Near schools
3. Near starbucks
4. Near restaurants with vegan options
5. Near a basketball court
6. Near a pet grooming services for the company dog

For all of these requirements we will use the Foursquare API. To avoid querying it excesively, all queries were done once and the data recovered from them saved in a DataFrame. We will import this data from an exported CSV. The detailed querying process can be seen in the foursquare.py Python Script that produced this file.